In [3]:
import os
import requests

import fitz
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

OPENROUTER_TOKEN = os.getenv("OPENROUTER_TOKEN")

In [4]:
def read_file_content(filepath):
    try:
        # Check if the file is a PDF
        if filepath.lower().endswith('.pdf'):
            doc = fitz.open(filepath)
            text = ""
            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                text += page.get_text()
            return text
        else:
            with open(filepath, 'r') as f:
                return f.read()
    except Exception as e:
        return f"Error reading file {filepath}: {e}"


# Read the guide and content files
guide_content = read_file_content("../doc/guide-to-itell-json.md")
uscis_content = read_file_content("../data/input-docs/split-pdfs/USCIS Chapter 1.pdf")
reference_json = read_file_content("../data/reference-json/uscis-one-nation-one-people-the-uscis-civics-test-textbook_page001_chapter-1-the-u-s-constitution.json")

In [ ]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

completion = client.chat.completions.create(
    extra_body={
        "provider": {
            "sort": "throughput"
        }
    },
    model="anthropic/claude-3.7-sonnet",
    messages=[
        {
          "role": "user",
          "content": [
              {
                  "type": "text",
                  "text": f"""Your Role: iTELL Content Authoring Expert
You are a specialized AI assistant expert in the iTELL framework. Your primary function is to convert source documents into perfectly structured iTELL JSON files.
Source Files for This Task:
The Guide:
{guide_content}

The Content:
{uscis_content}

Reference Output from Another Text:
{reference_json}

Primary Goal:
Your task is to analyze "The Content" file and generate a single, complete, and valid iTELL JSON file. You must adhere strictly and exclusively to the rules defined in "The Guide" file.
Final Output:
Generate the complete, valid JSON in a single code block. Do not include any explanatory text before or after the JSON block.
"""
              }
          ]
        }
    ]

)

print(completion.choices[0].message.content)